In [ ]:
#Libraries for the Project

import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
import numpy as np
from matplotlib import pyplot as plt
#from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

from sklearn.compose import ColumnTransformer

## Academic Data Analysis Capstone Project

INFO 698 Notebook
Created 10/7/2025

In [23]:
import pandas as pd
raw_data = pd.read_csv('DataSets/RawData.csv')

#print(raw_data.head())

# Data Load Preparation:
We wil clean and modify the data set in preparation for bulding our models

Load the base data set used for analysis and view the head

In [ ]:
#load dataset with null Term GPA excluded
data_set = pd.read_csv('DataSets/RawDataWithStudentsRemoved.csv')

print(data_set.head())

#data_set.dtypes
#data_set.shape

We review the count of null values for each feature of the data set.

In [ ]:

data_set_nulls = data_set.isnull().sum()
print(data_set_nulls)

We remove all rows (observations) where the target feature, *TermGPA* is NULL. 

We do not want to impute these missing values for our analysis.

In [52]:
data_set = data_set.dropna(subset = ['TermGPA'])

Below is acheck to confirm no NULL values remain in the data set.

In [ ]:
#no nulls
data_set_nulls = data_set.isnull().sum()
print(data_set_nulls)

Basic Statistics for the Data Set: Mean, Standard Deviation, and Quantiles

In [ ]:
#mean and standard dev by College

mean_by_college = data_set.groupby('College')['TermGPA'].mean()
print(mean_by_college)

stdev_by_college = data_set.groupby('College')['TermGPA'].std()
print(stdev_by_college)

In [17]:
# mean and quantiles

mean_termgpa = data_set['TermGPA'].mean()
print(mean_termgpa)


#75th percentile
q3_termgpa = np.percentile(data_set['TermGPA'], 50)

print(q3_termgpa)

3.1899030214688042
3.75


Test Train Split and Feature Inclusion for Data

We set the test train split here, and select which columns to exclude in the models for trainings.

In [ ]:
#Split the Data Set

# Y target column
target_column = 'TermGPA'
test_percent = 0.20


#data set dropping the target feature
x_features = data_set.drop(['FakeIdentifier' ,
                             #'College',
                             'CumulativeGPA',
                             'UnitsPassedincludedinGPA',
                             'UnitsPassednotincludedinGPA',
                             'TermGPA'],
                               axis = 1)


#encoding categorical features
columns_to_encode = ['Gender',
                      'PrimaryMilitaryAffiliation',
                        'AcademicCareer',
                          'College',
                      'UAFullTimePartTime',
                        'AcademicLevelEndofTerm',
                          'FirstGenerationFlag' 
                          ]

#define our target column on which we make predictions
y_target = data_set[target_column]


#split the data into a 80/20 train test split using the same random seed for consistency
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = test_percent, random_state = 698) 

#encode categorical features, and drop one level to avoid multicolinearity
x_train_encoded = pd.get_dummies(x_train,
                                  columns=columns_to_encode,
                                    drop_first=True,
                                      dtype = int)
x_test_encoded = pd.get_dummies(x_test,
                                 columns=columns_to_encode,
                                 drop_first=True,
                                   dtype = int)
# restore missing features after encoding
x_train_features_encoded_cols = x_train_encoded.columns
x_test_encoded = x_test_encoded.reindex(columns=x_train_features_encoded_cols, fill_value=0)

# K Nearest Neighbors Model Prep

In [ ]:
#Prepare both numeric and categorical features for scaling or encoding
numeric_features = x_train.select_dtypes(include=['number']).columns

categorical_features = x_train.select_dtypes(include=['category', 'object']).columns


In [55]:
# Scaler for data--KNN model, we need to scale the features for this model since KNN is sensitive to extreme values
scaler = StandardScaler()

In [56]:
#scale (normalize) and fit the numeric training data features
x_train_scaled_numeric = scaler.fit_transform(x_train[numeric_features])
x_train_scaled_numeric_df = pd.DataFrame(x_train_scaled_numeric, columns=numeric_features, index=x_train.index)

#apply same fit scaling to the test data numeric features
x_test_scaled_numeric = scaler.transform(x_test[numeric_features])
x_test_scaled_numeric_df = pd.DataFrame(x_test_scaled_numeric, columns=numeric_features, index=x_test.index)


In [ ]:
#one-hot encode the categorical features

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop = 'first')

#fit encoder on training data to ensure all training columns are accounted for when testing
encoder.fit(x_train[categorical_features])

#encode train data categorical features
x_train_encoded_categorical = encoder.fit_transform(x_train[categorical_features])
x_train_encoded_categorical_df = pd.DataFrame(x_train_encoded_categorical, columns=encoder.get_feature_names_out(categorical_features), index=x_train.index)

# Combine train processed features into a data frame
x_train_processed = pd.concat([x_train_scaled_numeric_df, x_train_encoded_categorical_df], axis=1)

#view the trainig data
print("Processed Training Data:")
print(x_train_processed.head())

#encode test data which is already fit to x train features-- do not fit again
x_test_encoded_categorical = encoder.transform(x_test[categorical_features])
x_test_encoded_categorical_df  = pd.DataFrame(x_test_encoded_categorical, columns=encoder.get_feature_names_out(categorical_features), index=x_test.index)

#combine test data (encoded categorical and the scaled numeric)
x_test_processed = pd.concat([x_test_scaled_numeric_df, x_test_encoded_categorical_df], axis=1)

# KNN Models
The data sets are encoded and scaled. They are ready for training and testing the KNN model.
We will perform a K nearest Neighbors Regression.

The initial k value was 5. 


Let's find an optimal k-value which gives the best MSE (and RMSE).

In [58]:
#list of k values, only odd to avoid ties

knn_odd_k_values = [kval for kval in range(1, 26) if kval % 2 != 0]

#loop through each kvalue for training and compare (R)MSE for test data
knn_mse_scores = []
for k in knn_odd_k_values:
    knn_regressor_bestk = KNeighborsRegressor(n_neighbors=k)
    knn_regressor_bestk.fit(x_train_processed, y_train)
    y_pred = knn_regressor_bestk.predict(x_test_processed)
    knn_mse_scores.append(mean_squared_error(y_test, y_pred))

# find and extract the best MSE value

print(min(knn_mse_scores))

best_k_index = np.argmin(knn_mse_scores)
best_k_value = knn_odd_k_values[best_k_index]
print(f"Optimal k value: {best_k_value}")
print(f"Minimum MSE: {knn_mse_scores[best_k_index]}")

1.1821020286718733
Optimal k value: 25
Minimum MSE: 1.1821020286718733


In [ ]:
# KNN Regressor Model
#Optimal k-value is 15, hardcoded here for efficiency
k_value = 25

# define knn model with number of neighbors
knn_regression_model = KNeighborsRegressor(n_neighbors = k_value)

#fit the knn model to the training data
knn_regression_model.fit(x_train_processed, y_train)

KNeighborsRegressor(n_neighbors=25)

In [ ]:
# predict for KNN on the test data

knn_y_pred = knn_regression_model.predict(x_test_processed)
#print(knn_y_pred)

# compare output of model with y test data (MSE)
knn_accuracy = mean_squared_error(y_test, knn_y_pred) 

# calculate the RMSE for KNN
knn_rmse = np.sqrt(knn_accuracy)

# Calculate the R-Squared for the KNN model
knn_rsquared = r2_score(y_test, knn_y_pred)

#View RMSE and R^2
print(f"Root Mean Squared Error (RMSE): {knn_rmse:.2f}")
print(f"R-squared (R2) Score: {knn_rsquared:.2f}")

[3.70936 3.90544 3.74192 ... 3.792   3.85    3.70952]
Root Mean Squared Error (RMSE): 1.09
R-squared (R2) Score: 0.14


### KNN Permutation Importance
We will review the permutation importnace for our KNN Regressor model.

## KNN Results

The KNN Regressor does not perform significantly different from the Random Forest Regressor Model. 

# Classifier Model Work

We define the features and parameters for the Random Forest Classifier Model. 

We set our *TermGPA* target column, define the test/train split percentage, select which features to remove from the model, and specify the categorical features which require a one-hot encoding for the RF model.

In [41]:

gpa_bins = [0, 2.0, 3.1, 3.4, 3.75, 4.0]
gpa_bin_labels = ['Warn', 'Pass', 'middle', 'Good', 'Excellent']

#binning the target into quartiles
#quantile_labels = ['Q1','Q2','Q3']

#y_train_binned, bins = pd.qcut(y_train, q=4, labels=False, retbins=True, duplicates = 'drop')


y_train_binned = pd.cut(y_train, bins=gpa_bins, include_lowest=True, labels=gpa_bin_labels)

#bin y test wiht the same bins as y train
y_test_binned = pd.cut(y_test, bins=gpa_bins, include_lowest=True, labels=gpa_bin_labels)

In [ ]:
#notnormally distributed

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis
from scipy.stats import skew
skew_termgpa = skew(data_set['TermGPA'])

kurtosis_termgpa = kurtosis(data_set['TermGPA'], axis=0, fisher=True, bias=True, nan_policy='propagate')

print(skew_termgpa)
print(kurtosis_termgpa)

sns.histplot(data_set['TermGPA'], kde=True)
plt.title('Histogram of TermGPA')
plt.show()

We set additional model parametrs and define the model as RFC. 

We use a consistent random seed parameter (698) where appropriate to ensure consistent behavior when training the model.

In [42]:
# the RF Calssifier model

metric = accuracy_score
metric_name = "accuracy"
rfc_depth = 20
rfc_max_features = None # None, sqrt or log2
rfc_min_sample_split = 10
#rfc_criterion = 'squared_error' #criterion{“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}, default=”squared_error”
rfc_estimators = 200

model = RandomForestClassifier(n_estimators = rfc_estimators,
                                 max_features = rfc_max_features,
                                    min_samples_split=rfc_min_sample_split,
                                      max_depth = rfc_depth,
                                          random_state=698)


We fit our trained RFC model to our test set. 

Then, we view the score of the model, based on our selected accuracy metric.

In [43]:
#fit the model
model.fit(x_train_encoded, y_train_binned)

# Predict using test data
y_pred = model.predict(x_test_encoded) 

    # compare prediction to actual dev/test data
score = metric(y_test_binned, y_pred)
print(f"{metric_name} on test set: {score:.4f}") 

#r_squared = r2_score(y_test, y_pred)
#print(f"r_squared on test set: {r_squared:.4f}") 

accuracy on test set: 0.5626


In [29]:
print(y_pred)

['Excellent' 'Excellent' 'Excellent' ... 'Excellent' 'Excellent'
 'Excellent']


In [ ]:
#feature importances:
feature_importances = model.feature_importances_ 
print(feature_importances)

feature_names = x_train_encoded.columns 
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
pd.set_option('display.max_rows', None)
print(feature_importance_df.sort_values(by = 'Importance', ascending = False))

In [ ]:
result = permutation_importance(model, x_test_encoded, y_test_binned, scoring = 'accuracy', n_repeats=10, random_state=698, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()

permutation_importances = pd.Series(result.importances_mean[sorted_idx], index=x_test_encoded.columns[sorted_idx]).sort_values(ascending=False)
print("\nPermutation Importance:")
print(permutation_importances)

In [ ]:
#processed_data = preprocessor.fit_transform(data_set)

# If you want to get a DataFrame back with named columns
# Get feature names after one-hot encoding
#ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
#all_feature_names = numeric_features + list(ohe_feature_names)

#processed_df = pd.DataFrame(processed_data, columns=all_feature_names)

With the optimal k-value extracted, we can run the KNN regressor model to find the RMSe and R Squared for our test set.